# 로또 번호 예측 프로젝트

### 데이터 로드

In [1]:
import pandas as pd

# df = pd.read_csv("lotto.csv", index_col=0) # 순서 미포함
df = pd.read_csv("lotto_ord.csv", index_col=0) # 순서 포함

최근 n개의 데이터만 사용

In [2]:
data = df[-200:]

1\~45의 번호를 0\~44로 변경

In [3]:
data = df - 1

당첨번호 리스트를 가공하여 sequences 생성

In [4]:
sequences = list()
for _, seq in data.iterrows():
    for i in range(1, len(seq)):
        sequence = list(seq)[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 6000


In [5]:
sequences[:5]

[[36, 22],
 [36, 22, 9],
 [36, 22, 9, 32],
 [36, 22, 9, 32, 28],
 [36, 22, 9, 32, 28, 39]]

잘린 sequences의 길이를 padding

In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

sequences = pad_sequences(sequences, maxlen=7, padding='pre')

In [7]:
import numpy as np

sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

print("X.shape:", X.shape)
print("y.shape:", y.shape)

X.shape: (6000, 6)
y.shape: (6000,)


### 모델 생성

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import Embedding

model = Sequential()
model.add(Embedding(45, 10))
model.add(LSTM(64, input_shape=(6, 1), return_sequences=False))
model.add(Dense(128, activation='relu'))
model.add(Dense(45, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics='acc')

In [9]:
model.fit(X, y, batch_size=64, epochs=200, verbose=1)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/200
188/188 [==============================] - 3s 5ms/step - loss: 3.8071 - acc: 0.0177
Epoch 2/200
188/188 [==============================] - 1s 5ms/step - loss: 3.8055 - acc: 0.0213
Epoch 3/200
188/188 [==============================] - 1s 5ms/step - loss: 3.8038 - acc: 0.0257
Epoch 4/200
188/188 [==============================] - 1s 5ms/step - loss: 3.7999 - acc: 0.0287
Epoch 5/200
188/188 [==============================] - 1s 5ms/step - loss: 3.7907 - acc: 0.0290
Epoch 6/200
188/188 [==============================] - 1s 5ms/step - loss: 3.7763 - acc: 0.0368
Epoch 7/200
188/188 [==============================] - 1s 5ms/step - loss: 3.7613 - acc: 0.0405
Epoch 8/200
188/188 [==============================] - 1s 5ms/step - loss: 3.7479 - acc: 0.0425
Epoch 9/200
188/188 [==============================] - 1s 5ms/step - loss: 3.7352 - acc: 0.048

188/188 [==============================] - 1s 5ms/step - loss: 1.5674 - acc: 0.6152
Epoch 82/200
188/188 [==============================] - 1s 5ms/step - loss: 1.5449 - acc: 0.6202
Epoch 83/200
188/188 [==============================] - 1s 5ms/step - loss: 1.5230 - acc: 0.6247
Epoch 84/200
188/188 [==============================] - 1s 5ms/step - loss: 1.5042 - acc: 0.6305
Epoch 85/200
188/188 [==============================] - 1s 5ms/step - loss: 1.4819 - acc: 0.6413
Epoch 86/200
188/188 [==============================] - 1s 5ms/step - loss: 1.4585 - acc: 0.6442
Epoch 87/200
188/188 [==============================] - 1s 5ms/step - loss: 1.4457 - acc: 0.6473
Epoch 88/200
188/188 [==============================] - 1s 5ms/step - loss: 1.4209 - acc: 0.6555
Epoch 89/200
188/188 [==============================] - 1s 5ms/step - loss: 1.4008 - acc: 0.6597
Epoch 90/200
188/188 [==============================] - 1s 5ms/step - loss: 1.3829 - acc: 0.6645
Epoch 91/200
188/188 [=====================

188/188 [==============================] - 1s 5ms/step - loss: 0.7500 - acc: 0.7862
Epoch 165/200
188/188 [==============================] - 1s 5ms/step - loss: 0.7716 - acc: 0.7820
Epoch 166/200
188/188 [==============================] - 1s 6ms/step - loss: 0.7716 - acc: 0.7762
Epoch 167/200
188/188 [==============================] - 2s 8ms/step - loss: 0.7479 - acc: 0.7827
Epoch 168/200
188/188 [==============================] - 2s 12ms/step - loss: 0.7315 - acc: 0.7878
Epoch 169/200
188/188 [==============================] - 1s 4ms/step - loss: 0.7262 - acc: 0.7882
Epoch 170/200
188/188 [==============================] - 1s 5ms/step - loss: 0.7219 - acc: 0.7883
Epoch 171/200
188/188 [==============================] - 1s 5ms/step - loss: 0.7206 - acc: 0.7885
Epoch 172/200
188/188 [==============================] - 1s 5ms/step - loss: 0.7263 - acc: 0.7858
Epoch 173/200
188/188 [==============================] - 1s 5ms/step - loss: 0.7197 - acc: 0.7858
Epoch 174/200
188/188 [==========

### 번호 리스트를 반환하는  함수

In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def seq_generate(model, input_num, verbose=False): 
    sequence = [input_num-1]

    while len(sequence) < 7:
        encoded = pad_sequences([sequence], maxlen=7, padding='pre')
        result = model.predict(encoded, verbose=0)
        
        mask = np.zeros(result.size, dtype=bool)
        mask[sequence] = True
        result = np.ma.array(result, mask=mask)
        result = np.argmax(result)

        sequence.append(result)
        
        if verbose:
            print("sequence:", np.array(sequence)+1)

    return sorted(np.array(sequence)+1)

테스트

In [11]:
test_num = 2
seq = [[test_num]]
encoded = pad_sequences(np.array(seq)-1, maxlen=7, padding='pre')
result = model.predict(encoded, verbose=0)

print(result)
print(np.argmax(result)+1)

seq_generate(model, test_num, verbose=True)

[[1.2949540e-02 1.9321748e-03 2.4985056e-03 1.2487484e-03 4.0442206e-02
  9.4558569e-03 6.7984395e-02 1.3989504e-01 2.7807003e-03 7.9197972e-04
  7.2272500e-04 1.2922476e-02 8.0416463e-03 1.7097041e-03 4.4813906e-03
  3.1554359e-01 1.2985866e-01 2.1443193e-04 3.1620029e-03 9.3130730e-05
  4.8668997e-04 6.6526090e-03 1.0750645e-03 3.9909240e-03 4.4504530e-03
  1.7782644e-04 2.0510727e-04 2.2922583e-02 1.3030976e-02 3.9272260e-02
  1.1879571e-03 5.5028684e-02 3.0150500e-03 1.2976717e-03 2.3423322e-02
  1.8905181e-03 1.6167250e-03 1.4956035e-04 2.0032080e-03 4.0857386e-02
  3.0083635e-03 1.2489057e-03 2.9365861e-04 7.0731412e-03 8.9124963e-03]]
16
sequence: [ 2 16]
sequence: [ 2 16  7]
sequence: [ 2 16  7 36]
sequence: [ 2 16  7 36 40]
sequence: [ 2 16  7 36 40 33]
sequence: [ 2 16  7 36 40 33 22]


[2, 7, 16, 22, 33, 36, 40]

### 수행 결과

첫번째로 나오는 번호 빈도순으로 출력

In [12]:
first_num = df['1'].value_counts()[:6]
first_num

37    35
25    31
26    31
23    29
4     28
45    28
Name: 1, dtype: int64

In [13]:
for num, _ in first_num.iteritems():
    print(f"{num:2}:", seq_generate(model, num))

37: [5, 9, 13, 15, 17, 37, 45]
25: [3, 8, 14, 16, 25, 33, 42]
26: [2, 8, 15, 25, 26, 41, 43]
23: [2, 19, 23, 25, 33, 35, 41]
 4: [4, 13, 26, 33, 37, 38, 40]
45: [2, 13, 17, 28, 39, 40, 45]


모든 숫자에 대해 결과 출력

In [14]:
for i in range(1, 46):
    print(f"{i:2}:", seq_generate(model, i))

 1: [1, 5, 11, 12, 18, 19, 41]
 2: [2, 7, 16, 22, 33, 36, 40]
 3: [3, 10, 16, 19, 23, 31, 42]
 4: [4, 13, 26, 33, 37, 38, 40]
 5: [5, 7, 19, 32, 34, 40, 41]
 6: [6, 10, 11, 32, 33, 40, 41]
 7: [3, 7, 10, 16, 25, 34, 38]
 8: [2, 8, 11, 19, 28, 42, 44]
 9: [8, 9, 10, 16, 24, 28, 40]
10: [3, 6, 10, 28, 34, 37, 43]
11: [5, 11, 12, 27, 33, 36, 42]
12: [12, 17, 33, 37, 39, 40, 44]
13: [13, 17, 28, 36, 40, 42, 44]
14: [6, 7, 12, 14, 17, 25, 31]
15: [8, 15, 18, 19, 31, 36, 40]
16: [13, 14, 16, 19, 23, 38, 45]
17: [1, 4, 6, 8, 17, 19, 42]
18: [6, 7, 17, 18, 37, 39, 45]
19: [5, 11, 12, 16, 18, 19, 41]
20: [14, 20, 21, 33, 34, 35, 40]
21: [9, 16, 19, 21, 31, 34, 36]
22: [1, 15, 18, 22, 23, 32, 43]
23: [2, 19, 23, 25, 33, 35, 41]
24: [7, 11, 13, 23, 24, 39, 41]
25: [3, 8, 14, 16, 25, 33, 42]
26: [2, 8, 15, 25, 26, 41, 43]
27: [11, 19, 23, 27, 34, 35, 45]
28: [11, 16, 19, 28, 29, 42, 45]
29: [4, 7, 23, 29, 34, 36, 44]
30: [2, 7, 13, 25, 30, 33, 35]
31: [12, 16, 25, 26, 27, 31, 43]
32: [8, 12, 14, 2